In [1]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys
fp = codecs.open("toji.txt", "r", encoding="utf-16")
soup = BeautifulSoup(fp, "html.parser")
body = soup.select_one("body")
text = body.getText() + " "
print('코퍼스의 길이: ', len(text))


/Users/yun/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


코퍼스의 길이:  311682


In [2]:

# 문자를 하나하나 읽어 들이고 ID 붙이기
chars = sorted(list(set(text)))
print('사용되고 있는 문자의 수:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars)) # 문자 → ID
indices_char = dict((i, c) for i, c in enumerate(chars)) # ID → 문자

사용되고 있는 문자의 수: 1692


In [3]:
# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록하기
maxlen = 20 # 한 문장의 길이가 maxlen
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
    
print('학습할 구문의 수:', len(sentences))
print('텍스트를 ID 벡터로 변환합니다...')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) #문장 수, 문장길이, 단어(글자) 수
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) # 문장 수, 단어 수

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    


학습할 구문의 수: 103888
텍스트를 ID 벡터로 변환합니다...


In [4]:

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
# 모델 구축하기(LSTM)
print('모델을 구축합니다...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars)))) #문장길이, 단어수
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

모델을 구축합니다...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               932352    
_________________________________________________________________
dense_1 (Dense)              (None, 1692)              218268    
_________________________________________________________________
activation_1 (Activation)    (None, 1692)              0         
Total params: 1,150,620
Trainable params: 1,150,620
Non-trainable params: 0
_________________________________________________________________


In [6]:

# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 3):
    print()
    print('-' * 50)
    print('반복 =', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1) # 
    
    
    # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('--- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('--- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        # 시드를 기반으로 텍스트 자동 생성
        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            # 예측한 문자를 시드로 생성한 후보를 배열에서 꺼내기
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
반복 = 1
Epoch 1/1
   128/103888 [..............................] - ETA: 1:23 - loss: 3.7656

/Users/yun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


103888/103888 [==============================] - 77s 743us/step - loss: 3.5806

--- 다양성 =  0.2
--- 시드 = "를 뻗은 소나무 아래로 자리를 옮겨간"
를 뻗은 소나무 아래로 자리를 옮겨간 것도 없었다. 그러나 그래도 그 말이 없었다. 그래도 그래도 그래도 그 일 아니다. 그는 그래도 없는 것이 없었다. 그러나 그래도 그래도 그래도 아니다. 그래도 아니다. 그래도 그래도 없었다. 그러나 그래도 그러나 그래도 없는 것도 아니 것도 없었다. 그러나 그래도 그래도 없었다. 그 말 사는 아니다. 그래도 그러나 그래도 없었다. 그러나 그래도 사람이 없는 것도 없었다. 그래도 그래도 그래도 그래도 그래 그래도 그래도 그래 아니다. 그래도 그래도 그래도 그래도 그 말 가지. 그러나 그 자신 일이 있었다. 그러나 그래도 그래도 그래도 그래도 그 자신 없었다. 그래 그래도 그 아니 것도 아니다. 그래도 그래도 그 말이 없었다. 그래도 이 없는 것도 없었다. 그래도 없었다. 아니 것도 아니다. 그러나 그래 그

--- 다양성 =  0.5
--- 시드 = "를 뻗은 소나무 아래로 자리를 옮겨간"
를 뻗은 소나무 아래로 자리를 옮겨간다. 그는 없었다.
"나? 그래도 못 불어야 하겄소. 그 말 그러나 그런 병이 있었다. 한 기래도 그러믄 아니가. 마을 사람이  기이 없었다.
"아니 요."
"무, 그래도 들어서 부자리 그 아니지."
"아, 없소. 그놈은 손으로 애기씨."
"다시 말이 아니고. 그러나 그렇고, 이 그래 못 되어 있었다. 죽는 것도 아니다. 그래는 이놈의 얼굴이 지르는 것이다. 그러나 비마을 내가 무신 가지 않았다. 그 말이나 가서 있는 것도 아니다. 그러나 그 울리 얼굴에 들어 있었다. 아이들은 아니다. 그런  정이 없었다. 그러나  나서 서방에 그 말은 아니께 그래도 사람이 없었다. 그 일 아니었다. 아니는 사람들이 없어야 하는 것 같은 하는 눈을 두만네는 난 몸이 웃었다. 사나이 그래 그래도 